En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

Antes de comenzar con la ejecucion de las funciones, es necesario que las dependencias necesarias esten instaladas. Se recomienda crear un entorno virtual con los paquetes definidos en `requirements.txt` y ejecutar este Jupyter Notebook dentro de ese entorno.

In [1]:
# pip install memory-profiler==0.61.0

### Pregunta 1

1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns: 
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```

In [157]:
from typing import List, Tuple
import datetime
import json
import os
import pandas as pd


def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Opening JSON file
    df = pd.read_json(path_or_buf=file_path, lines=True)
        

fp = "farmers-protest-tweets-2021-2-4.json"
q1_time(fp)

Ya que el archivo .json tiene varias lineas con objetos json, usamos el modulo de `Pandas` `read_json` que nos permite leer el archivo por linea.

In [236]:
df = pd.read_json(path_or_buf=file_path, lines=True)

Ya que los valores de la columna `user` son un objeto json, procedemos a extraer el campo `username`.

In [237]:
df['user'].iloc[0]

{'username': 'ArjunSinghPanam',
 'displayname': 'Arjun Singh Panam',
 'id': 45091142,
 'description': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie',
 'rawDescription': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie',
 'descriptionUrls': [],
 'verified': False,
 'created': '2009-06-06T07:50:57+00:00',
 'followersCount': 603,
 'friendsCount': 311,
 'statusesCount': 17534,
 'favouritesCount': 4269,
 'listedCount': 23,
 'mediaCount': 1211,
 'location': '',
 'protected': False,
 'linkUrl': 'https://www.cosmosmovieofficial.com',
 'linkTcourl': 'https://t.co/3uaoV3gCt3',
 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1215541746492461056/3De61YoQ_normal.jpg',
 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/45091142/1612601766',
 'url': 'https://twit

In [238]:
# Normalizar atributo 'username'
df['user'] = pd.json_normalize(df.user)['username']

Tambien necesitamos usar solo los valores de año, mes y dia de la columna `date`.

In [239]:
df['date'].iloc[0]

Timestamp('2021-02-24 09:23:35+0000', tz='UTC')

In [240]:
df['date'] = df['date'].dt.date
df['date'].iloc[0]

datetime.date(2021, 2, 24)

Agrupar por fecha y contar filas.

In [241]:
df2 = df[:]

group_by = ['date']
limit = 10


df2 = df2[['date', 'user']]
df2 = df2.groupby('date').count()
df = df.sort_values('user', ascending=False)
df2 = df2.iloc[:limit, :]

df2 = df2.reset_index()

dates = df2['date'].to_list()
dates

[datetime.date(2021, 2, 12),
 datetime.date(2021, 2, 13),
 datetime.date(2021, 2, 14),
 datetime.date(2021, 2, 15),
 datetime.date(2021, 2, 16),
 datetime.date(2021, 2, 17),
 datetime.date(2021, 2, 18),
 datetime.date(2021, 2, 19),
 datetime.date(2021, 2, 20),
 datetime.date(2021, 2, 21)]

In [242]:
### Ahora top usuario por dia
df2 = df[:]

top_users = []
for date in dates:
    df2 = df[:]
    where = df2['date'] == date
    group_by = ['user']
    limit = 10

    df2 = df2[where]
    df2 = df2[['date', 'user']]
    df2 = df2.groupby(group_by).count()
    df2 = df2.sort_values('date', ascending=False)
    df2 = df2.iloc[:limit, :]
    
    top_users.append(df2.index[0])
    
top_users

['RanbirS00614606',
 'MaanDee08215437',
 'rebelpacifist',
 'jot__b',
 'jot__b',
 'RaaJVinderkaur',
 'neetuanjle_nitu',
 'Preetm91',
 'MangalJ23056160',
 'Surrypuria']

In [243]:
final_list = []
for i in range(10):
    final_list.append((dates[i], top_users[i]))
    
final_list

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 21), 'Surrypuria')]